In [2]:
import numpy as np
import pandas as pd
import re
import mapply
# set mapply
mapply.init(n_workers=-1, progressbar=True)

import nltk
nltk.download('punkt')
import string
from nltk.corpus import stopwords
import json

/home/claudia/HLT/project/code/hlt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/claudia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Read the dataset
lyrics = pd.read_csv('lyrics.csv')
lyrics

,id,title,artist,lyrics,tag
0,7,Lollipop Remix,Lil Wayne,[Intro: Lil Wayne]\nHaha\nUh-huh\nNo homo (You...,rap
1,19,Losing Weight Pt. 2,Cam'ron,"[Chorus: Cam'ron]\nAyo, fuck losing weight\nI'...",rap
2,26,A Milli,Fabolous,It's time to get money\nI got a million reason...,rap
3,29,Warning,The Notorious B.I.G.,[Produced by Easy Mo Bee]\n\n[Verse 1: The Not...,rap
4,40,On My Own,Lil Wayne,"[Intro: Lil Wayne]\nYeah, hit me with the snar...",rap
...,...,...,...,...,...
84211,7864996,SUHO - Grey Suit English Translation,Genius English Translations,"[Verse 1]\nI feel faded, I'm not used to this ...",rock
84212,7865871,HEARTFIRST,Kelsea Ballerini,[Verse 1]\nMet him at a party\nAccidentally br...,country
84213,7868429,Never Had a Chance,Katherine Li,[Chorus]\nWhy am I still here and waiting for ...,pop
84214,7871730,Red Alert Boss Metal Zone,Clutch,[Verse 1]\nI used to work for Doc Tyrell\nIt d...,rock


In [3]:
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84216 entries, 0 to 84215
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      84216 non-null  int64 
 1   title   84213 non-null  object
 2   artist  84216 non-null  object
 3   lyrics  84216 non-null  object
 4   tag     84216 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.2+ MB


In [4]:
# Check for missing values
lyrics.isna().sum()

id        0
title     3
artist    0
lyrics    0
tag       0
dtype: int64

In [5]:
# Drop rows with missing values
lyrics = lyrics.dropna(subset='title')
lyrics.isna().sum()

id        0
title     0
artist    0
lyrics    0
tag       0
dtype: int64

In [40]:
for tag in lyrics['tag'].unique():
    tag_perc = len(lyrics[lyrics['tag'] == tag]) / len(lyrics)
    print(f'{tag}: {tag_perc: .1f}')

pop:  0.2
rock:  0.2
rap:  0.2
rb:  0.2
country:  0.2


In [6]:
# Lyrics contain meta information between square brackets
print(lyrics['lyrics'][0])

[Intro: Lil Wayne]
Haha
Uh-huh
No homo (Young Mula, baby!)
I say, he's so sweet, make her wanna lick the wrapper
Remix, baby!

[Verse 1: Kanye West]
Lollipop, lollipop, breastses just like Dolly Parton
She ride my spaceship 'til she hit the top
That hit the spot
'Til she ask, "How many li-i-li-i-licks do it take" 'til she get to shop?
Don't worry why my wrists got so freeze
Tell a girl, "Like Doritos, that's not 'cho cheese"
Tell her friends, "Like Fritos, I'm tryin' to lay"
I can't only have one, and I ain't trying to wait
This a song with Wayne, so you know it's gon' melt
But you ain't finna murder me like everybody else
I'ma rap like I got some type respect for myself
I don't do it for my health, man, I do it for the belt
Man, I do it to the death, 'til the roof get melt
Hundred degrees, drop the roof, so the coupe don't melt
Man, the flow so cold, chicken soup won't help
We need four more hoes, we need oh, oh, oh, oh!
You know what it is when we out of town
We balling too serious, 

In [7]:
# Clean lyrics (remove content btw square brackets)
del_sqbrackets = re.compile(r'\[.*?\]')
lyrics['cleaned_lyrics'] = lyrics['lyrics'].mapply(lambda x: del_sqbrackets.sub('', x))
lyrics = lyrics.drop(columns=['lyrics'])
lyrics = lyrics.rename(columns={'cleaned_lyrics': 'lyrics'})
print(lyrics['lyrics'][0])

100%|██████████| 40/40 [00:02<00:00, 13.93it/s]
/tmp/ipykernel_135481/3727505202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lyrics['cleaned_lyrics'] = lyrics['lyrics'].mapply(lambda x: del_sqbrackets.sub('', x))



Haha
Uh-huh
No homo (Young Mula, baby!)
I say, he's so sweet, make her wanna lick the wrapper
Remix, baby!


Lollipop, lollipop, breastses just like Dolly Parton
She ride my spaceship 'til she hit the top
That hit the spot
'Til she ask, "How many li-i-li-i-licks do it take" 'til she get to shop?
Don't worry why my wrists got so freeze
Tell a girl, "Like Doritos, that's not 'cho cheese"
Tell her friends, "Like Fritos, I'm tryin' to lay"
I can't only have one, and I ain't trying to wait
This a song with Wayne, so you know it's gon' melt
But you ain't finna murder me like everybody else
I'ma rap like I got some type respect for myself
I don't do it for my health, man, I do it for the belt
Man, I do it to the death, 'til the roof get melt
Hundred degrees, drop the roof, so the coupe don't melt
Man, the flow so cold, chicken soup won't help
We need four more hoes, we need oh, oh, oh, oh!
You know what it is when we out of town
We balling too serious, and you out of bounds
So come here, bab

In [8]:
# misc lyrics category contain unexpected items
misc_lyrics = lyrics[lyrics['tag'] == 'misc']
misc_lyrics['title'].to_list()

['Penny for a Thought',
 'John 1',
 'John 3',
 'Im Nobody Who Are You?',
 'Buffalo Bills',
 'Do Not Go Gentle Into That Good Night',
 'The Second Coming',
 'Ego-Tripping there may be a reason',
 'The Animal Soul',
 'The Love Song of J. Alfred Prufrock',
 'Kubla Khan',
 'Mending Wall',
 'The Mower Against The Gardens',
 'The Road Not Taken',
 'The Tyger',
 'The Lamb',
 'Singing In The Rain',
 'Andak Andak',
 'Genesis 1',
 'Ecclesiastes 1',
 'Sonnet 18',
 'Baba Says Cool for Thought',
 'Stopping by Woods on a Snowy Evening',
 'Who Will Survive America',
 'Comment 1',
 'Nicki Minaj',
 'The Emperor of Ice Cream',
 'He Wishes for the Cloths of Heaven',
 'Introduction / The Revolution Will Not Be Televised Small Talk at 125th and Lenox Version',
 'The Bill of Rights',
 'Jabberwocky',
 'And Did Those Feet In Ancient Time',
 'The Rime Of The Ancient Mariner',
 'Psalm 23',
 'Matthew 25',
 'Is This Really What You Want?',
 'To His Coy Mistress',
 'Revelation 1',
 'I Have a Dream',
 'Mentos Theme

In [9]:
print('Examples:\n')
print('\n2012 Presidential Acceptance Speech----------------------------')
print(misc_lyrics[misc_lyrics['title'] == '2012 Presidential Acceptance Speech']['lyrics'].item()[0:800])
print('\nTreaty of Versailles Articles 231 - 247----------------------------')
print(misc_lyrics[misc_lyrics['title'] == 'Treaty of Versailles Articles 231 - 247']['lyrics'].item()[0:800])
print('\nHansel and Gretel----------------------------')
print(misc_lyrics[misc_lyrics['title'] == 'Hansel and Gretel']['lyrics'].item()[0:800])
print('\nThe Picture of Dorian Gray Preface----------------------------')
print(misc_lyrics[misc_lyrics['title'] == 'The Picture of Dorian Gray Preface']['lyrics'].item()[0:800])

Examples:


2012 Presidential Acceptance Speech----------------------------
Thank you so much. Tonight, more than 200 years after a former colony won the right to determine its own destiny, the task of perfecting our union moves forward. It moves forward because of you. It moves forward because you reaffirmed the spirit that has triumphed over war and depression, the spirit that has lifted this country from the depths of despair to the great heights of hope, the belief that while each of us will pursue our own individual dreams, we are an American family and we rise or fall together as one nation and as one people.

Tonight, in this election, you, the American people, reminded us that while our road has been hard, while our journey has been long, we have picked ourselves up, we have fought our way back, and we know in our hearts that for the United States of Ameri

Treaty of Versailles Articles 231 - 247----------------------------
Reparations

PART VIII

SECTION I

GENERAL PROVISIONS


In [10]:
# Drop all rows with the 'misc' tag
lyrics = lyrics.loc[lyrics['tag'] != 'misc']
lyrics

,id,title,artist,tag,lyrics
0,7,Lollipop Remix,Lil Wayne,rap,"\nHaha\nUh-huh\nNo homo (Young Mula, baby!)\nI..."
1,19,Losing Weight Pt. 2,Cam'ron,rap,"\nAyo, fuck losing weight\nI'm back on these h..."
2,26,A Milli,Fabolous,rap,It's time to get money\nI got a million reason...
3,29,Warning,The Notorious B.I.G.,rap,"\n\n\nWho the fuck is this, pagin' me at 5:46 ..."
4,40,On My Own,Lil Wayne,rap,"\nYeah, hit me with the snares, Fresh\nYeah-ye..."
...,...,...,...,...,...
84211,7864996,SUHO - Grey Suit English Translation,Genius English Translations,rock,"\nI feel faded, I'm not used to this new color..."
84212,7865871,HEARTFIRST,Kelsea Ballerini,country,\nMet him at a party\nAccidentally brushed his...
84213,7868429,Never Had a Chance,Katherine Li,pop,\nWhy am I still here and waiting for us to ha...
84214,7871730,Red Alert Boss Metal Zone,Clutch,rock,\nI used to work for Doc Tyrell\nIt didn't end...


In [11]:
lyrics.drop_duplicates()

,id,title,artist,tag,lyrics
0,7,Lollipop Remix,Lil Wayne,rap,"\nHaha\nUh-huh\nNo homo (Young Mula, baby!)\nI..."
1,19,Losing Weight Pt. 2,Cam'ron,rap,"\nAyo, fuck losing weight\nI'm back on these h..."
2,26,A Milli,Fabolous,rap,It's time to get money\nI got a million reason...
3,29,Warning,The Notorious B.I.G.,rap,"\n\n\nWho the fuck is this, pagin' me at 5:46 ..."
4,40,On My Own,Lil Wayne,rap,"\nYeah, hit me with the snares, Fresh\nYeah-ye..."
...,...,...,...,...,...
84211,7864996,SUHO - Grey Suit English Translation,Genius English Translations,rock,"\nI feel faded, I'm not used to this new color..."
84212,7865871,HEARTFIRST,Kelsea Ballerini,country,\nMet him at a party\nAccidentally brushed his...
84213,7868429,Never Had a Chance,Katherine Li,pop,\nWhy am I still here and waiting for us to ha...
84214,7871730,Red Alert Boss Metal Zone,Clutch,rock,\nI used to work for Doc Tyrell\nIt didn't end...


In [12]:
lyrics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70178 entries, 0 to 84215
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      70178 non-null  int64 
 1   title   70178 non-null  object
 2   artist  70178 non-null  object
 3   tag     70178 non-null  object
 4   lyrics  70178 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.2+ MB


In [15]:
# Save the cleaned dataset
lyrics.to_csv('cleaned_lyrics.csv', index=False)

## Data analysis

In [4]:
lyrics = pd.read_csv('cleaned_lyrics.csv')
lyrics['length'] = lyrics['lyrics'].mapply(lambda x: len(x))

100%|██████████| 40/40 [00:00<00:00, 42.94it/s]


In [5]:
mean_lengths = []
max_lengths = []
min_lengths = []

for tag in lyrics['tag'].unique():
    tag_lengths = lyrics[lyrics['tag'] == tag]['length']

    mean_len = tag_lengths.mean()
    max_len = tag_lengths.max()
    min_len = tag_lengths.min()
    
    mean_lengths.append(mean_len)
    max_lengths.append(max_len)
    min_lengths.append(min_len)

    print(f'{tag} -----------------------------------')
    print(f'mean length: {mean_len}\nmax length: {max_len}\nmin length: {min_len}')

rock -----------------------------------
mean length: 1190.5120769504808
max length: 12661
min length: 101
pop -----------------------------------
mean length: 1361.075591336563
max length: 46201
min length: 100
rb -----------------------------------
mean length: 1654.6956893480585
max length: 73878
min length: 101
country -----------------------------------
mean length: 1295.5785836420632
max length: 16283
min length: 100
rap -----------------------------------
mean length: 2673.737674551154
max length: 79626
min length: 112


In [6]:
lyrics['tokens'] = lyrics['lyrics'].mapply(lambda x: nltk.word_tokenize(x))
lyrics['n_tokens'] = lyrics['tokens'].mapply(lambda x: len(x))

mean_lengths = []
max_lengths = []
min_lengths = []

for tag in lyrics['tag'].unique():
    tag_lengths = lyrics[lyrics['tag'] == tag]['n_tokens']

    mean_len = tag_lengths.mean()
    max_len = tag_lengths.max()
    min_len = tag_lengths.min()

    mean_lengths.append(mean_len)
    max_lengths.append(max_len)
    min_lengths.append(min_len)

    print(f'{tag} -----------------------------------')
    print(f'mean length: {mean_len}\nmax length: {max_len}\nmin length: {min_len}')

100%|██████████| 40/40 [01:15<00:00,  1.88s/it]


rock -----------------------------------
mean length: 267.00263626647666
max length: 2881
min length: 20
pop -----------------------------------
mean length: 315.6692077514962
max length: 7787
min length: 18
rb -----------------------------------
mean length: 402.37691485571787
max length: 18087
min length: 21
country -----------------------------------
mean length: 294.30443146195495
max length: 3674
min length: 24
rap -----------------------------------
mean length: 627.5619834710744
max length: 16129
min length: 24


In [7]:
# Load English stopwords
stop_words = set(stopwords.words('english'))

def process_tokens(tokens):
    tokens = [token.lower() for token in tokens]  # Convert tokens to lowercase
    tokens = [token for token in tokens if token.isalpha()]  # Remove non-alphabetic tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords
    tokens = [''.join(char for char in token if char not in string.punctuation) for token in tokens]  # Remove punctuation
    tokens = [token for token in tokens if token.strip() != ""]  # Remove empty tokens
    return tokens

lyrics['filtered_tokens'] = lyrics['tokens'].mapply(process_tokens)

100%|██████████| 40/40 [01:28<00:00,  2.20s/it]


In [9]:
lyrics['filtered_tokens']

0        [could, hold, hold, icecaps, melting, transist...
1        [nothing, could, ask, nothing, need, world, ba...
2        [gon, pop, xanny, brown, skin, fatty, dasani, ...
3        [heroes, gone, come, great, land, america, sti...
4        [cheated, lied, said, love, cheated, lied, sai...
                               ...                        
70173    [another, song, reasons, stressed, melody, get...
70174    [yeah, people, rapper, right, could, ever, var...
70175    [hollygrove, go, hollywood, uhh, hollygrove, h...
70176    [seen, men, look, think, see, like, think, mak...
70177    [lay, awake, night, oh, low, troubled, get, jo...
Name: filtered_tokens, Length: 70178, dtype: object

In [10]:
lyrics['n_filtered_tokens'] = lyrics['filtered_tokens'].mapply(lambda x: len(x))

mean_lengths = []
max_lengths = []
min_lengths = []

for tag in lyrics['tag'].unique():
    tag_lengths = lyrics[lyrics['tag'] == tag]['n_filtered_tokens']

    mean_len = tag_lengths.mean()
    max_len = tag_lengths.max()
    min_len = tag_lengths.min()

    mean_lengths.append(mean_len)
    max_lengths.append(max_len)
    min_lengths.append(min_len)

    print(f'{tag} -----------------------------------')
    print(f'mean length: {mean_len}\nmax length: {max_len}\nmin length: {min_len}')

100%|██████████| 40/40 [00:33<00:00,  1.19it/s]


rock -----------------------------------
mean length: 112.55311720698255
max length: 1230
min length: 6
pop -----------------------------------
mean length: 130.9160729552579
max length: 3914
min length: 6
rb -----------------------------------
mean length: 161.3851086569291
max length: 7009
min length: 4
country -----------------------------------
mean length: 125.64826161299516
max length: 1577
min length: 5
rap -----------------------------------
mean length: 275.18509546879454
max length: 7051
min length: 8


In [11]:
lyrics['filtered_unique_tokens'] = lyrics['filtered_tokens'].mapply(lambda x: list(set(x)))

100%|██████████| 40/40 [00:39<00:00,  1.02it/s]


In [12]:
lyrics['filtered_unique_tokens']

0        [lost, want, window, could, little, burning, p...
1        [could, say, jesa, hard, asked, beliya, seen, ...
2        [trust, go, want, us, always, brown, neck, sit...
3        [last, fair, nontax, want, land, read, us, put...
4        [oh, keep, nothing, lied, try, somethng, cheat...
                               ...                        
70173    [last, light, go, lead, want, song, reason, ca...
70174    [diaper, uh, go, want, respect, could, putting...
70175    [hahahaha, pt, go, range, gun, californicate, ...
70176    [look, drinkin, friend, tell, single, actin, h...
70177    [oh, generation, even, decent, go, lie, baby, ...
Name: filtered_unique_tokens, Length: 70178, dtype: object

In [13]:
lyrics['n_filtered_unique_tokens'] = lyrics['filtered_unique_tokens'].mapply(lambda x: len(x))

mean_lengths = []
max_lengths = []
min_lengths = []

for tag in lyrics['tag'].unique():
    tag_lengths = lyrics[lyrics['tag'] == tag]['n_filtered_unique_tokens']

    mean_len = tag_lengths.mean()
    max_len = tag_lengths.max()
    min_len = tag_lengths.min()

    mean_lengths.append(mean_len)
    max_lengths.append(max_len)
    min_lengths.append(min_len)

    print(f'{tag} -----------------------------------')
    print(f'mean length: {mean_len}\nmax length: {max_len}\nmin length: {min_len}')

100%|██████████| 40/40 [00:19<00:00,  2.09it/s]


rock -----------------------------------
mean length: 59.746704666904165
max length: 498
min length: 2
pop -----------------------------------
mean length: 60.81575947563408
max length: 1594
min length: 1
rb -----------------------------------
mean length: 69.7464196651229
max length: 1741
min length: 2
country -----------------------------------
mean length: 67.34212026218296
max length: 700
min length: 4
rap -----------------------------------
mean length: 156.57566258193216
max length: 2987
min length: 2


In [14]:
lyrics

,id,title,artist,tag,lyrics,length,tokens,n_tokens,filtered_tokens,n_filtered_tokens,filtered_unique_tokens,n_filtered_unique_tokens
0,359018,Gamma Ray,Beck,rock,\nIf I could hold\nHold out for now\nWith thes...,724,"[If, I, could, hold, Hold, out, for, now, With...",140,"[could, hold, hold, icecaps, melting, transist...",72,"[lost, want, window, could, little, burning, p...",48
1,3610742,Beliya,The Vamps,pop,Nothing more I could ask for\nNothing more tha...,1419,"[Nothing, more, I, could, ask, for, Nothing, m...",297,"[nothing, could, ask, nothing, need, world, ba...",176,"[could, say, jesa, hard, asked, beliya, seen, ...",39
2,4007667,Titanic,FRVRFRIDAY,rb,\nShe gon pop a xanny\nBrown skin with a fatty...,1559,"[She, gon, pop, a, xanny, Brown, skin, with, a...",368,"[gon, pop, xanny, brown, skin, fatty, dasani, ...",157,"[trust, go, want, us, always, brown, neck, sit...",107
3,937615,Where Have All Our Heroes Gone,Bill Anderson,country,Where have all our heroes gone\nWhat's come ov...,2864,"[Where, have, all, our, heroes, gone, What, 's...",587,"[heroes, gone, come, great, land, america, sti...",266,"[last, fair, nontax, want, land, read, us, put...",202
4,1523825,You Cheated,The Shields (Soul),pop,"You cheated, you lied\nYou said that you love ...",474,"[You, cheated, ,, you, lied, You, said, that, ...",118,"[cheated, lied, said, love, cheated, lied, sai...",43,"[oh, keep, nothing, lied, try, somethng, cheat...",17
...,...,...,...,...,...,...,...,...,...,...,...,...
70173,758875,My Story,Addison Road,pop,This is not another song about the reasons I a...,1070,"[This, is, not, another, song, about, the, rea...",242,"[another, song, reasons, stressed, melody, get...",87,"[last, light, go, lead, want, song, reason, ca...",57
70174,81170,8 Mile Road,Jon Connor,rap,\nYeah\nThe People's Rapper (That's right) Tha...,3744,"[Yeah, The, People, 's, Rapper, (, That, 's, r...",857,"[yeah, people, rapper, right, could, ever, var...",357,"[diaper, uh, go, want, respect, could, putting...",199
70175,685108,HollyWeezy,Lil Wayne,rap,\nToo Hollygrove to go Hollywood\nUhh\n\n\nHol...,4316,"[Too, Hollygrove, to, go, Hollywood, Uhh, Holl...",912,"[hollygrove, go, hollywood, uhh, hollygrove, h...",419,"[hahahaha, pt, go, range, gun, californicate, ...",247
70176,1390290,Shes Actin Single Im Drinkin Doubles,Gary Stewart,country,\nI've seen men look at her before\nAnd they t...,875,"[I, 've, seen, men, look, at, her, before, And...",208,"[seen, men, look, think, see, like, think, mak...",74,"[look, drinkin, friend, tell, single, actin, h...",44


In [15]:
for tag in lyrics['tag'].unique():
    tag_perc = len(lyrics[lyrics['tag'] == tag]) / len(lyrics)
    print(f'{tag}: {tag_perc: .1f}')

rock:  0.2
pop:  0.2
rb:  0.2
country:  0.2
rap:  0.2


In [16]:
# Save the augmented dataset
lyrics['tokens'] = lyrics['tokens'].apply(json.dumps)
lyrics['filtered_tokens'] = lyrics['filtered_tokens'].apply(json.dumps)
lyrics['filtered_unique_tokens'] = lyrics['filtered_unique_tokens'].apply(json.dumps)
lyrics.to_csv('augmented_lyrics.csv', index=False)

## Zero-shot classification

In [18]:
lyrics = pd.read_csv('augmented_lyrics.csv')
lyrics['tokens'] = lyrics['tokens'].apply(json.loads)
lyrics['filtered_tokens'] = lyrics['filtered_tokens'].apply(json.loads)
lyrics['filtered_unique_tokens'] = lyrics['filtered_unique_tokens'].apply(json.loads)

In [23]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="valhalla/distilbart-mnli-12-3")
classifier(
    "This is a tutorial about the Transformers library in HuggingFace",
    candidate_labels=["politics", "education", "business"],
)

{'sequence': 'This is a tutorial about the Transformers library in HuggingFace',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.9194583296775818, 0.04611356183886528, 0.034428082406520844]}

In [30]:
type(lyrics['lyrics'].to_list())
type(list(lyrics['tag'].unique()))

list

In [31]:
predictions = classifier(
    lyrics['lyrics'].to_list(),
    candidate_labels=lyrics['tag'].unique(),
)

predictions

KeyboardInterrupt: 

In [25]:
len(predictions)

3

## Data splitting

In [19]:
lyrics = lyrics.sample(frac=1, replace=False).reset_index(drop=True)
nrows = len(lyrics)
split1 = int(0.6 * nrows)
split2 = int(0.8 * nrows)
train_set = lyrics.iloc[:split1].reset_index(drop=True)
val_set = lyrics.iloc[split1:split2].reset_index(drop=True)
test_set = lyrics.iloc[split2:].reset_index(drop=True)
print('\nTRAINING SET:')
print(train_set.info())
print('\nVALIDATION SET:')
print(val_set.info())
print('\nTEST SET:')
print(test_set.info())


TRAINING SET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42106 entries, 0 to 42105
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        42106 non-null  int64 
 1   title                     42106 non-null  object
 2   artist                    42106 non-null  object
 3   tag                       42106 non-null  object
 4   lyrics                    42106 non-null  object
 5   length                    42106 non-null  int64 
 6   tokens                    42106 non-null  object
 7   n_tokens                  42106 non-null  int64 
 8   filtered_tokens           42106 non-null  object
 9   n_filtered_tokens         42106 non-null  int64 
 10  filtered_unique_tokens    42106 non-null  object
 11  n_filtered_unique_tokens  42106 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 3.9+ MB
None

VALIDATION SET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex

In [20]:
train_set

,id,title,artist,tag,lyrics,length,tokens,n_tokens,filtered_tokens,n_filtered_tokens,filtered_unique_tokens,n_filtered_unique_tokens
0,681241,The Cape,Guy Clark,country,\nEight years old with a flour sack cape\nTied...,1312,"[Eight, years, old, with, a, flour, sack, cape...",289,"[eight, years, old, flour, sack, cape, tied, a...",126,"[sure, trust, tied, years, screwed, arms, fing...",71
1,6115912,​heartbreak of america,lovelytheband,rock,\nThis is the heartbreak\nThis is the heartbre...,1784,"[This, is, the, heartbreak, This, is, the, hea...",389,"[heartbreak, heartbreak, heartbreak, wipe, fee...",175,"[steppеd, gun, freedom, land, us, bruv, reason...",93
2,3277190,I Am Who I Am,Baka Not Nice,rap,"\nNiggas wanna hang, yeah (hang, nigga)\nCause...",2486,"[Niggas, wan, na, hang, ,, yeah, (, hang, ,, n...",721,"[niggas, wan, na, hang, yeah, hang, nigga, cau...",267,"[last, go, want, two, hang, trend, pipes, stas...",88
3,3007450,Drive,Chris Travis,rap,"\nWater, water\nFuck a damn job, Imma' get min...",1500,"[Water, ,, water, Fuck, a, damn, job, ,, Imma,...",391,"[water, water, fuck, damn, job, imma, get, min...",181,"[came, bro, hi, yuh, liked, huh, really, real,...",109
4,6013803,One More Hallelujah,Brennley Brown,country,"\nSometimes, life ain't easy\nSometimes, you'r...",1696,"[Sometimes, ,, life, ai, n't, easy, Sometimes,...",330,"[sometimes, life, ai, easy, sometimes, praying...",156,"[go, window, could, thankful, staying, whisper...",79
...,...,...,...,...,...,...,...,...,...,...,...,...
42101,2456105,Hey hey yeah,RavenEye,rock,\n\nVerse 1:\nYes I know that she's no good\nO...,1554,"[Verse, 1, :, Yes, I, know, that, she, 's, no,...",380,"[verse, yes, know, good, gon, na, leave, pain,...",180,"[good, even, look, oooohohhhhhooooh, one, want...",73
42102,99502,Hilikus,Incubus,rock,\nAbout a hundred years ago now\nThought I was...,1552,"[About, a, hundred, years, ago, now, Thought, ...",342,"[hundred, years, ago, thought, left, dead, sol...",121,"[block, years, good, go, friend, could, knowle...",57
42103,1789633,Check,Fetty Wap & DRiivE,rap,\nBaby call me Xoovier thats my name (thats my...,3072,"[Baby, call, me, Xoovier, thats, my, name, (, ...",690,"[baby, call, xoovier, thats, name, thats, name...",344,"[go, wash, want, links, beamer, could, respect...",154
42104,449564,Family Tradition,Hank Williams Jr.,country,\nCountry music singers have always been a rea...,1442,"[Country, music, singers, have, always, been, ...",332,"[country, music, singers, always, real, close,...",127,"[smoke, unique, although, guess, want, others,...",74


In [21]:
val_set

,id,title,artist,tag,lyrics,length,tokens,n_tokens,filtered_tokens,n_filtered_tokens,filtered_unique_tokens,n_filtered_unique_tokens
0,3338936,What Went Wrong,Rusty Machines,rock,\nWell looked how this turned out for the both...,714,"[Well, looked, how, this, turned, out, for, th...",159,"[well, looked, turned, us, decided, stay, forg...",60,"[different, sorry, us, well, came, much, said,...",36
1,37298,Crownin Me,Playa Fly,rap,Nigga's ask me.. you know what I'm saying..\nI...,3592,"[Nigga, 's, ask, me, .., you, know, what, I, '...",776,"[nigga, ask, know, saying, ever, reach, top, f...",382,"[slugs, smoke, flizy, shelter, always, suckas,...",243
2,2272377,Chartered Trips,Hsker D,rock,I packed up my belongings\nIn a nylon carry-al...,1528,"[I, packed, up, my, belongings, In, a, nylon, ...",335,"[packed, belongings, nylon, hear, porter, call...",150,"[go, guess, trees, call, nylon, said, ever, ho...",39
3,6135525,Good Girl,Charlotte Cardin,pop,\nYou spent all my money playing poker with yo...,1102,"[You, spent, all, my, money, playing, poker, w...",285,"[spent, money, playing, poker, buddies, mind, ...",99,"[oh, good, fake, let, buddies, tell, forgive, ...",52
4,3093833,Heaven Sent // Hell Bound,Vanish,rock,The wolves in the street they start to retreat...,1798,"[The, wolves, in, the, street, they, start, to...",391,"[wolves, street, start, retreat, sunshine, awa...",167,"[last, surely, smoke, go, putting, well, death...",117
...,...,...,...,...,...,...,...,...,...,...,...,...
14031,1638232,If You Say Go,Rita Springer,pop,"Playlist: Sweetly Broken album version:\nOh, y...",1273,"[Playlist, :, Sweetly, Broken, album, version,...",333,"[playlist, sweetly, broken, album, version, oh...",117,"[oh, good, look, withdraw, go, album, water, u...",31
14032,1302888,Dusty Old Dust So Long Its Been Good to Know Yuh,Woody Guthrie,country,"I've sung this song, but I'll sing it again\nO...",1336,"[I, 've, sung, this, song, ,, but, I, 'll, sin...",323,"[sung, song, sing, place, lived, wild, windy, ...",126,"[last, us, read, song, could, dust, jumped, si...",86
14033,103771,Lounge Fly,Stone Temple Pilots,rock,"\nIt's okay, it's okay\nOkay, believe me\nPins...",1722,"[It, 's, okay, ,, it, 's, okay, Okay, ,, belie...",446,"[okay, okay, okay, believe, pins, pins, kill, ...",153,"[let, want, empty, always, kill, live, said, o...",41
14034,707764,No One Understands,Bayside,rock,\nI've always played the fool around here\nI'm...,1627,"[I, 've, always, played, the, fool, around, he...",358,"[always, played, fool, around, starting, worry...",147,"[oh, one, always, could, felt, best, much, foo...",73


In [22]:
test_set

,id,title,artist,tag,lyrics,length,tokens,n_tokens,filtered_tokens,n_filtered_tokens,filtered_unique_tokens,n_filtered_unique_tokens
0,896842,Next Ex Kick Em Out,Beyonc,rb,\nWoah-ho ooh\n\n\nI know that you treat me ri...,2108,"[Woah-ho, ooh, I, know, that, you, treat, me, ...",521,"[ooh, know, treat, right, still, think, got, t...",231,"[oh, last, tied, woah, look, move, sorry, boy,...",66
1,100615,Stand,Sly and the Family Stone,rb,\nStand\nIn the end you'll still be you\nOne t...,768,"[Stand, In, the, end, you, 'll, still, be, you...",164,"[stand, end, still, one, done, things, set, st...",68,"[things, go, crease, one, want, fall, deal, cr...",43
2,306029,Roots Rock Reggae,Bob Marley & The Wailers,rock,\nPlay I some music (This a Reggae music)\nPla...,1190,"[Play, I, some, music, (, This, a, Reggae, mus...",275,"[play, music, reggae, music, play, music, regg...",112,"[sure, good, want, one, refuse, play, hundred,...",28
3,3035542,All In,Empire Cast,rb,"\nYou and I, you and I, had our time, had our ...",1355,"[You, and, I, ,, you, and, I, ,, had, our, tim...",345,"[time, time, moments, start, fade, away, baby,...",119,"[last, trust, adore, go, us, cup, budge, two, ...",79
4,695880,Go To Dumb,Lil B,rock,\nIt's your boy Lil B\nBased freestyle in New ...,2274,"[It, 's, your, boy, Lil, B, Based, freestyle, ...",509,"[boy, lil, b, based, freestyle, new, york, cit...",244,"[smoke, go, want, song, stay, blunts, pretty, ...",85
...,...,...,...,...,...,...,...,...,...,...,...,...
14031,2986575,God I’m Sorry,Boosie Badazz & C-Murder,rap,"\nC Rhythm \n\n\nFirst off, I wanna apologize ...",3293,"[C, Rhythm, First, off, ,, I, wan, na, apologi...",833,"[c, rhythm, first, wan, na, apologize, every, ...",302,"[uh, keeping, destined, much, forgiveness, sta...",191
14032,1262205,I Still Love You,Next,rb,"\nI still love you, babe\n(It ain't ever gonna...",2553,"[I, still, love, you, ,, babe, (, It, ai, n't,...",693,"[still, love, babe, ai, ever, gon, na, change,...",237,"[prove, last, oh, even, let, one, want, always...",68
14033,3176659,Queen of the New World,Pre Kai Ro,rb,It’s what you doin’ oh no \n\nSad royalty girl...,2573,"[It, ’, s, what, you, doin, ’, oh, no, Sad, ro...",671,"[doin, oh, sad, royalty, girl, dont, spoil, gi...",214,"[go, want, song, taking, cool, still, codeine,...",82
14034,28150,Don’t Mess wit Texas,Lil Keke,rap,"\nPut your stones in dust and worn, South coas...",3638,"[Put, your, stones, in, dust, and, worn, ,, So...",835,"[put, stones, dust, worn, south, coast, raised...",402,"[ill, trust, uh, playahaters, everythang, keke...",239


In [23]:
for df, s in zip([train_set, val_set, test_set], ['Training set', 'Validation set', 'Test set']):
    print(f'\n{s}:')
    for tag in df['tag'].unique():
        tag_perc = len(df[df['tag'] == tag]) / len(df)
        print(f'{tag}: {tag_perc: .1f}')


Training set:
country:  0.2
rock:  0.2
rap:  0.2
rb:  0.2
pop:  0.2

Validation set:
rock:  0.2
rap:  0.2
pop:  0.2
country:  0.2
rb:  0.2

Test set:
rb:  0.2
rock:  0.2
rap:  0.2
country:  0.2
pop:  0.2


In [36]:
# Save the splits
for df, s in zip([train_set, val_set, test_set], ['train_set', 'val_set', 'test_set']):
    df['tokens'] = df['tokens'].apply(json.dumps)
    df['filtered_tokens'] = df['filtered_tokens'].apply(json.dumps)
    df['filtered_unique_tokens'] = df['filtered_unique_tokens'].apply(json.dumps)
    df.to_csv(f'{s}.csv', index=False)

## Models

In [1]:
import numpy as np
import pandas as pd
import mapply
# set mapply
mapply.init(n_workers=-1, progressbar=True)

import json

/home/claudia/HLT/project/code/hlt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load training set
train_set = pd.read_csv('train_set.csv')

train_set['tokens'] = train_set['tokens'].apply(json.loads)
train_set['filtered_tokens'] = train_set['filtered_tokens'].apply(json.loads)
train_set['filtered_unique_tokens'] = train_set['filtered_unique_tokens'].apply(json.loads)

# Load validation set
val_set = pd.read_csv('val_set.csv')

val_set['tokens'] = val_set['tokens'].apply(json.loads)
val_set['filtered_tokens'] = val_set['filtered_tokens'].apply(json.loads)
val_set['filtered_unique_tokens'] = val_set['filtered_unique_tokens'].apply(json.loads)

In [3]:
import torch
import torchtext.vocab as vocab

# Load FastText embeddings
fasttext_embeddings = vocab.FastText('simple')

def vectorize(lyric_tokens):
    lyric_tokens = [token for token in lyric_tokens if token in fasttext_embeddings.stoi]
    vectors = [fasttext_embeddings[token] for token in lyric_tokens]
    lyric_vector = torch.mean(torch.stack(vectors), dim=0)
    return torch.tensor(lyric_vector)

# Compute FastText embeddings
train_set['FastText_embedding'] = train_set['filtered_tokens'].mapply(vectorize)
val_set['FastText_embedding'] = val_set['filtered_tokens'].mapply(vectorize)

  0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_8843/2334475650.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(lyric_vector)
/tmp/ipykernel_8843/2334475650.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(lyric_vector)
/tmp/ipykernel_8843/2334475650.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(lyric_vector)
  2%|▎         | 1/40 [00:01<01:14,  1.90s/it]/tmp/ipykernel_8843/2334475650.py:11: UserWarning: To copy construct from a tensor, it is recommended to u

In [4]:
len(train_set)

42106

In [5]:
len(train_set['FastText_embedding'][0])

300

In [4]:
label = {
    'rap': torch.tensor(0, dtype=torch.long),
    'rb': torch.tensor(1, dtype=torch.long),
    'pop': torch.tensor(2, dtype=torch.long),
    'rock': torch.tensor(3, dtype=torch.long),
    'country': torch.tensor(4, dtype=torch.long)
}

train_set['label'] = train_set['tag'].mapply(lambda x: label[x])
val_set['label'] = val_set['tag'].mapply(lambda x: label[x])

100%|██████████| 40/40 [00:00<00:00, 50.14it/s]


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

class LyricsDataset(Dataset):
    def __init__(self, lyrics_df):
        
        self.x = train_set['FastText_embedding'].values
        self.y = train_set['label'].values
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

training_set = LyricsDataset(train_set)
train_dataloader = DataLoader(training_set, batch_size=50000, shuffle=True)

validation_set = LyricsDataset(val_set)
val_dataloader = DataLoader(validation_set, batch_size=50000, shuffle=True)

In [6]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, 1000),#128
            nn.ReLU(),
            nn.Linear(1000, output_dim),
            #nn.ReLU(),
            #nn.Linear(300, output_dim),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

model = NeuralNetwork(input_dim = 300, output_dim = 5)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training loop
epochs = 1000

model.train()
for epoch in range(epochs):
    for batch_x, batch_y in train_dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/1000], Loss: 1.6115
Epoch [2/1000], Loss: 1.6002
Epoch [3/1000], Loss: 1.5905
Epoch [4/1000], Loss: 1.5804
Epoch [5/1000], Loss: 1.5697
Epoch [6/1000], Loss: 1.5583
Epoch [7/1000], Loss: 1.5463
Epoch [8/1000], Loss: 1.5335
Epoch [9/1000], Loss: 1.5200
Epoch [10/1000], Loss: 1.5059
Epoch [11/1000], Loss: 1.4912
Epoch [12/1000], Loss: 1.4760
Epoch [13/1000], Loss: 1.4606
Epoch [14/1000], Loss: 1.4451
Epoch [15/1000], Loss: 1.4297
Epoch [16/1000], Loss: 1.4144
Epoch [17/1000], Loss: 1.3996
Epoch [18/1000], Loss: 1.3853
Epoch [19/1000], Loss: 1.3718
Epoch [20/1000], Loss: 1.3590
Epoch [21/1000], Loss: 1.3472
Epoch [22/1000], Loss: 1.3362
Epoch [23/1000], Loss: 1.3260
Epoch [24/1000], Loss: 1.3166
Epoch [25/1000], Loss: 1.3079
Epoch [26/1000], Loss: 1.2998
Epoch [27/1000], Loss: 1.2922
Epoch [28/1000], Loss: 1.2850
Epoch [29/1000], Loss: 1.2782
Epoch [30/1000], Loss: 1.2717
Epoch [31/1000], Loss: 1.2654
Epoch [32/1000], Loss: 1.2594
Epoch [33/1000], Loss: 1.2536
Epoch [34/1000], Lo

In [8]:
train_predictions = []
train_targets = []

model.eval()

size = len(train_dataloader.dataset)
num_batches = len(train_dataloader)
loss, correct = 0, 0

with torch.no_grad():
    for X, y in train_dataloader:
        pred = model(X)
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_predictions.append(pred.argmax(1))
        train_targets.append(y)
        
loss /= num_batches
correct /= size
print(f"Training Error: \n Accuracy: {(correct):>0.2f}, Avg loss: {loss:>8f} \n")

Training Error: 
 Accuracy: 0.76, Avg loss: 0.698524 



In [9]:
val_predictions = []
val_targets = []

model.eval()

size = len(val_dataloader.dataset)
num_batches = len(val_dataloader)
loss, correct = 0, 0

with torch.no_grad():
    for X, y in val_dataloader:
        pred = model(X)
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        val_predictions.append(pred.argmax(1))
        val_targets.append(y)

loss /= num_batches
correct /= size
print(f"Validation Error: \n Accuracy: {(correct):>0.2f}, Avg loss: {loss:>8f} \n")

Validation Error: 
 Accuracy: 0.76, Avg loss: 0.698523 



In [10]:
train_predictions = train_predictions[0].tolist()
train_targets = train_targets[0].tolist()

train_predictions = [x for x in train_predictions]
train_targets = [x for x in train_targets]

val_predictions = val_predictions[0].tolist()
val_targets = val_targets[0].tolist()

val_predictions = [x for x in val_predictions]
val_targets = [x for x in val_targets]

In [11]:
def evaluate(predictions, targets, num_classes):
    """
    Compute evaluation metrics for multiclass classification tasks: accuracy, precision, recall, and F1-score.

    Args:
        predictions (list): Predicted class labels (sequence_length).
        targets (list): True class labels (sequence_length).
        num_classes (int): Number of classes in the classification task.

    Returns:
        dict: Dictionary containing evaluation metrics: accuracy, precision, recall, and F1-score.
    """
    # Ensure predictions and targets have the same length
    assert len(predictions) == len(targets)
    
    # Convert lists to numpy arrays if they are not already
    predictions = np.array(predictions)
    targets = np.array(targets)
    
    # Initialize evaluation metrics arrays for each class
    true_positives = np.zeros(num_classes)
    true_negatives = np.zeros(num_classes)
    false_positives = np.zeros(num_classes)
    false_negatives = np.zeros(num_classes)
    
    # Compute true positives, false positives, false negatives, and class counts for each class
    for i in range(num_classes):
        true_positives[i] = np.sum((predictions == i) & (targets == i))
        true_negatives[i] = np.sum((predictions != i) & (targets != i))
        false_positives[i] = np.sum((predictions == i) & (targets != i))
        false_negatives[i] = np.sum((predictions != i) & (targets == i))
    
    # Compute accuracy
    accuracy = np.mean(predictions == targets)
    
    # Compute precision, recall, and F1-score for each class
    precisions = true_positives / (true_positives + false_positives)
    recalls = true_positives / (true_positives + false_negatives)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
    accuracies = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
    
    # Compute micro-averaged precision, recall, and F1-score
    micro_precision = np.sum(true_positives) / np.sum(true_positives + false_positives)
    micro_recall = np.sum(true_positives) / np.sum(true_positives + false_negatives)
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)
    
    # Compute macro-averaged precision, recall, and F1-score
    macro_precision = np.mean(precisions)
    macro_recall = np.mean(recalls)
    macro_f1 = np.mean(f1_scores)
    
    # Return evaluation metrics as dictionary
    evaluation_metrics = {
        'extended_metrics':{
            'true_positives': true_positives,
            'true_negatives': true_negatives,
            'false_positives': false_positives,
            'false_negatives': false_negatives,
            'precision': precisions,
            'recall': recalls,
            'f1_score': f1_scores,
            'accuracy': accuracies
        },
        'aggregated_metrics':{
            'accuracy': accuracy,
            'micro_precision': micro_precision,
            'micro_recall': micro_recall,
            'micro_f1': micro_f1,
            'macro_precision': macro_precision,
            'macro_recall': macro_recall,
            'macro_f1': macro_f1
        }
    }
    
    return evaluation_metrics

In [12]:
train_evaluation_results = evaluate(predictions=train_predictions, targets=train_targets, num_classes=5)
val_evaluation_results = evaluate(predictions=val_predictions, targets=val_targets, num_classes=5)

In [13]:
for metric, value in train_evaluation_results['aggregated_metrics'].items():
    print(f"{metric.capitalize()}: {value:.4f}")

print('\n')

for metric, value in val_evaluation_results['aggregated_metrics'].items():
    print(f"{metric.capitalize()}: {value:.4f}")

Accuracy: 0.7560
Micro_precision: 0.7560
Micro_recall: 0.7560
Micro_f1: 0.7560
Macro_precision: 0.7554
Macro_recall: 0.7558
Macro_f1: 0.7550


Accuracy: 0.7560
Micro_precision: 0.7560
Micro_recall: 0.7560
Micro_f1: 0.7560
Macro_precision: 0.7554
Macro_recall: 0.7558
Macro_f1: 0.7550


In [15]:
for i in range(5):
    print(f'\nClass {i}')
    for metric, value in train_evaluation_results['extended_metrics'].items():
        print(f'{metric.capitalize()}: {value[i]:.4f}')


Class 0
True_positives: 7204.0000
True_negatives: 32416.0000
False_positives: 1284.0000
False_negatives: 1202.0000
Precision: 0.8487
Recall: 0.8570
F1_score: 0.8528
Accuracy: 0.9410

Class 1
True_positives: 6340.0000
True_negatives: 31029.0000
False_positives: 2655.0000
False_negatives: 2082.0000
Precision: 0.7048
Recall: 0.7528
F1_score: 0.7280
Accuracy: 0.8875

Class 2
True_positives: 5290.0000
True_negatives: 31578.0000
False_positives: 2169.0000
False_negatives: 3069.0000
Precision: 0.7092
Recall: 0.6329
F1_score: 0.6689
Accuracy: 0.8756

Class 3
True_positives: 6289.0000
True_negatives: 31558.0000
False_positives: 2085.0000
False_negatives: 2174.0000
Precision: 0.7510
Recall: 0.7431
F1_score: 0.7470
Accuracy: 0.8989

Class 4
True_positives: 6709.0000
True_negatives: 31569.0000
False_positives: 2081.0000
False_negatives: 1747.0000
Precision: 0.7633
Recall: 0.7934
F1_score: 0.7780
Accuracy: 0.9091


In [16]:
for i in range(5):
    print(f'\nClass {i}')
    for metric, value in val_evaluation_results['extended_metrics'].items():
        print(f'{metric.capitalize()}: {value[i]:.4f}')


Class 0
True_positives: 7204.0000
True_negatives: 32416.0000
False_positives: 1284.0000
False_negatives: 1202.0000
Precision: 0.8487
Recall: 0.8570
F1_score: 0.8528
Accuracy: 0.9410

Class 1
True_positives: 6340.0000
True_negatives: 31029.0000
False_positives: 2655.0000
False_negatives: 2082.0000
Precision: 0.7048
Recall: 0.7528
F1_score: 0.7280
Accuracy: 0.8875

Class 2
True_positives: 5290.0000
True_negatives: 31578.0000
False_positives: 2169.0000
False_negatives: 3069.0000
Precision: 0.7092
Recall: 0.6329
F1_score: 0.6689
Accuracy: 0.8756

Class 3
True_positives: 6289.0000
True_negatives: 31558.0000
False_positives: 2085.0000
False_negatives: 2174.0000
Precision: 0.7510
Recall: 0.7431
F1_score: 0.7470
Accuracy: 0.8989

Class 4
True_positives: 6709.0000
True_negatives: 31569.0000
False_positives: 2081.0000
False_negatives: 1747.0000
Precision: 0.7633
Recall: 0.7934
F1_score: 0.7780
Accuracy: 0.9091


## LSTM

In [17]:
import numpy as np
import pandas as pd
import mapply
import torch
# set mapply
mapply.init(n_workers=-1, progressbar=True)

import json
# Load training set
train_set = pd.read_csv('train_set.csv')

train_set['tokens'] = train_set['tokens'].apply(json.loads)
train_set['filtered_tokens'] = train_set['filtered_tokens'].apply(json.loads)
train_set['filtered_unique_tokens'] = train_set['filtered_unique_tokens'].apply(json.loads)

# Load validation set
val_set = pd.read_csv('val_set.csv')

val_set['tokens'] = val_set['tokens'].apply(json.loads)
val_set['filtered_tokens'] = val_set['filtered_tokens'].apply(json.loads)
val_set['filtered_unique_tokens'] = val_set['filtered_unique_tokens'].apply(json.loads)

label = {
    'rap': torch.tensor(0, dtype=torch.long),
    'rb': torch.tensor(1, dtype=torch.long),
    'pop': torch.tensor(2, dtype=torch.long),
    'rock': torch.tensor(3, dtype=torch.long),
    'country': torch.tensor(4, dtype=torch.long)
}

train_set['label'] = train_set['tag'].mapply(lambda x: label[x])
val_set['label'] = val_set['tag'].mapply(lambda x: label[x])

100%|██████████| 40/40 [00:00<00:00, 47.72it/s]


In [18]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torchtext.vocab as vocab

# Load FastText embeddings
fasttext_embeddings = vocab.FastText('simple')

def vectorize_2(lyric_tokens):
    lyric_tokens = lyric_tokens[0:5]
    lyric_tokens = [token for token in lyric_tokens if token in fasttext_embeddings.stoi]
    lyrics_vector = [fasttext_embeddings[token].tolist() for token in lyric_tokens]
    return lyrics_vector

# Compute FastText embeddings
train_set['FastText_embedding_2'] = train_set['filtered_tokens'].apply(vectorize_2)
val_set['FastText_embedding_2'] = val_set['filtered_tokens'].apply(vectorize_2)

In [19]:
sequences = [torch.tensor(seq) for seq in train_set['FastText_embedding_2'].to_list() if len(seq) != 0]
labels = [label for seq, label in zip(train_set['FastText_embedding_2'].to_list(), train_set['label'].to_list()) if len(seq) != 0]

In [20]:

from torch.utils.data import Dataset, DataLoader

class LyricsDataset(Dataset):
    def __init__(self, lyrics_df):
        
        self.x = sequences
        self.y = labels
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def custom_collate_fn(batch):
    # Sort batch based on sequence length for pack_padded_sequence
    batch.sort(key=lambda x: len(x[0]), reverse=True)
    sequences, labels = zip(*batch)
    
    # Pad sequences to the length of the longest sequence in the batch
    padded_sequences = pad_sequence(sequences, batch_first=True)
    
    # Compute sequence lengths for pack_padded_sequence
    seq_lengths = torch.tensor([len(seq) for seq in sequences])
    
    return padded_sequences, labels, seq_lengths

training_set = LyricsDataset(train_set)
train_dataloader = DataLoader(training_set, batch_size=50000, shuffle=True, collate_fn=custom_collate_fn)

validation_set = LyricsDataset(val_set)
val_dataloader = DataLoader(validation_set, batch_size=50000, shuffle=True, collate_fn=custom_collate_fn)

In [21]:
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = int(hidden_size/2)
        self.lstm = nn.LSTM(input_size, int(hidden_size/2), batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, seq_lengths):
        packed_input = pack_padded_sequence(input_seq, seq_lengths, batch_first=True, enforce_sorted=True)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.fc(output[:, -1, :])  # Get the last output of each sequence
        return output

# Example parameters
embedding_dim = 300  # Dimension of word embeddings
hidden_dim = 128  # Hidden dimension of the LSTM
output_dim = 5  # Number of output classes

# Initialize the model
model = LSTMModel(embedding_dim, hidden_dim, output_dim)

# loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
epochs = 100

model.train()
for epoch in range(epochs):
    for X, y, seq_lengths in train_dataloader:
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)

        optimizer.zero_grad()

        outputs = model(X, seq_lengths)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

/tmp/ipykernel_8843/1923054973.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


Epoch [1/100], Loss: 1.6130
Epoch [2/100], Loss: 1.6085
Epoch [3/100], Loss: 1.6047
Epoch [4/100], Loss: 1.6014
Epoch [5/100], Loss: 1.5985
Epoch [6/100], Loss: 1.5958
Epoch [7/100], Loss: 1.5932
Epoch [8/100], Loss: 1.5905
Epoch [9/100], Loss: 1.5877
Epoch [10/100], Loss: 1.5847
Epoch [11/100], Loss: 1.5816
Epoch [12/100], Loss: 1.5782
Epoch [13/100], Loss: 1.5746
Epoch [14/100], Loss: 1.5708
Epoch [15/100], Loss: 1.5668
Epoch [16/100], Loss: 1.5626
Epoch [17/100], Loss: 1.5583
Epoch [18/100], Loss: 1.5538
Epoch [19/100], Loss: 1.5492
Epoch [20/100], Loss: 1.5445
Epoch [21/100], Loss: 1.5397
Epoch [22/100], Loss: 1.5349
Epoch [23/100], Loss: 1.5302
Epoch [24/100], Loss: 1.5257
Epoch [25/100], Loss: 1.5214
Epoch [26/100], Loss: 1.5174
Epoch [27/100], Loss: 1.5137
Epoch [28/100], Loss: 1.5103
Epoch [29/100], Loss: 1.5071
Epoch [30/100], Loss: 1.5039
Epoch [31/100], Loss: 1.5007
Epoch [32/100], Loss: 1.4974
Epoch [33/100], Loss: 1.4942
Epoch [34/100], Loss: 1.4910
Epoch [35/100], Loss: 1

In [22]:
loss_fn = nn.CrossEntropyLoss()
train_predictions = []
train_targets = []

model.eval()

size = len(train_dataloader.dataset)
num_batches = len(train_dataloader)
loss, correct = 0, 0

with torch.no_grad():
    for X, y, seq_lengths in train_dataloader:
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        pred = model(X, seq_lengths)
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_predictions.append(pred.argmax(1))
        train_targets.append(y)
        
loss /= num_batches
correct /= size
print(f"Training Error: \n Accuracy: {(correct):>0.2f}, Avg loss: {loss:>8f} \n")

/tmp/ipykernel_8843/4234377109.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


Training Error: 
 Accuracy: 0.43, Avg loss: 1.385330 



In [23]:
loss_fn = nn.CrossEntropyLoss()
val_predictions = []
val_targets = []

model.eval()

size = len(val_dataloader.dataset)
num_batches = len(val_dataloader)
loss, correct = 0, 0

with torch.no_grad():
    for X, y, seq_lengths in val_dataloader:
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        pred = model(X, seq_lengths)
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        val_predictions.append(pred.argmax(1))
        val_targets.append(y)

loss /= num_batches
correct /= size
print(f"Validation Error: \n Accuracy: {(correct):>0.2f}, Avg loss: {loss:>8f} \n")

/tmp/ipykernel_8843/1500241406.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


Validation Error: 
 Accuracy: 0.43, Avg loss: 1.385330 



In [24]:
train_predictions = train_predictions[0].tolist()
train_targets = train_targets[0].tolist()

train_predictions = [x for x in train_predictions]
train_targets = [x for x in train_targets]

val_predictions = val_predictions[0].tolist()
val_targets = val_targets[0].tolist()

val_predictions = [x for x in val_predictions]
val_targets = [x for x in val_targets]

train_evaluation_results = evaluate(predictions=train_predictions, targets=train_targets, num_classes=5)
val_evaluation_results = evaluate(predictions=val_predictions, targets=val_targets, num_classes=5)

In [25]:
for metric, value in train_evaluation_results['aggregated_metrics'].items():
    print(f"{metric.capitalize()}: {value:.4f}")

print('\n')

for metric, value in val_evaluation_results['aggregated_metrics'].items():
    print(f"{metric.capitalize()}: {value:.4f}")

Accuracy: 0.4292
Micro_precision: 0.4292
Micro_recall: 0.4292
Micro_f1: 0.4292
Macro_precision: 0.4185
Macro_recall: 0.4289
Macro_f1: 0.4179


Accuracy: 0.4292
Micro_precision: 0.4292
Micro_recall: 0.4292
Micro_f1: 0.4292
Macro_precision: 0.4185
Macro_recall: 0.4289
Macro_f1: 0.4179


In [26]:
for i in range(5):
    print(f'\nClass {i}')
    for metric, value in train_evaluation_results['extended_metrics'].items():
        print(f'{metric.capitalize()}: {value[i]:.4f}')


Class 0
True_positives: 5451.0000
True_negatives: 27992.0000
False_positives: 5682.0000
False_negatives: 2938.0000
Precision: 0.4896
Recall: 0.6498
F1_score: 0.5584
Accuracy: 0.7951

Class 1
True_positives: 2944.0000
True_negatives: 29288.0000
False_positives: 4365.0000
False_negatives: 5466.0000
Precision: 0.4028
Recall: 0.3501
F1_score: 0.3746
Accuracy: 0.7663

Class 2
True_positives: 1867.0000
True_negatives: 30237.0000
False_positives: 3477.0000
False_negatives: 6482.0000
Precision: 0.3494
Recall: 0.2236
F1_score: 0.2727
Accuracy: 0.7632

Class 3
True_positives: 3529.0000
True_negatives: 28458.0000
False_positives: 5144.0000
False_negatives: 4932.0000
Precision: 0.4069
Recall: 0.4171
F1_score: 0.4119
Accuracy: 0.7605

Class 4
True_positives: 4261.0000
True_negatives: 28266.0000
False_positives: 5343.0000
False_negatives: 4193.0000
Precision: 0.4437
Recall: 0.5040
F1_score: 0.4719
Accuracy: 0.7733


In [27]:
for i in range(5):
    print(f'\nClass {i}')
    for metric, value in val_evaluation_results['extended_metrics'].items():
        print(f'{metric.capitalize()}: {value[i]:.4f}')


Class 0
True_positives: 5451.0000
True_negatives: 27992.0000
False_positives: 5682.0000
False_negatives: 2938.0000
Precision: 0.4896
Recall: 0.6498
F1_score: 0.5584
Accuracy: 0.7951

Class 1
True_positives: 2944.0000
True_negatives: 29288.0000
False_positives: 4365.0000
False_negatives: 5466.0000
Precision: 0.4028
Recall: 0.3501
F1_score: 0.3746
Accuracy: 0.7663

Class 2
True_positives: 1867.0000
True_negatives: 30237.0000
False_positives: 3477.0000
False_negatives: 6482.0000
Precision: 0.3494
Recall: 0.2236
F1_score: 0.2727
Accuracy: 0.7632

Class 3
True_positives: 3529.0000
True_negatives: 28458.0000
False_positives: 5144.0000
False_negatives: 4932.0000
Precision: 0.4069
Recall: 0.4171
F1_score: 0.4119
Accuracy: 0.7605

Class 4
True_positives: 4261.0000
True_negatives: 28266.0000
False_positives: 5343.0000
False_negatives: 4193.0000
Precision: 0.4437
Recall: 0.5040
F1_score: 0.4719
Accuracy: 0.7733


## RNN

In [28]:
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# LSTM model
class RNNodel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = int(hidden_size/2)
        self.lstm = nn.RNN(input_size, int(hidden_size/2), batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, seq_lengths):
        packed_input = pack_padded_sequence(input_seq, seq_lengths, batch_first=True, enforce_sorted=True)
        packed_output, _ = self.lstm(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.fc(output[:, -1, :])  # Get the last output of each sequence
        return output

# Example parameters
embedding_dim = 300  # Dimension of word embeddings
hidden_dim = 128  # Hidden dimension of the LSTM
output_dim = 5  # Number of output classes

# Initialize the model
model = LSTMModel(embedding_dim, hidden_dim, output_dim)

# loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
epochs = 100

model.train()
for epoch in range(epochs):
    for X, y, seq_lengths in train_dataloader:
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        
        

        optimizer.zero_grad()

        outputs = model(X, seq_lengths)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

/tmp/ipykernel_8843/2227556864.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


Epoch [1/100], Loss: 1.6115
Epoch [2/100], Loss: 1.6080
Epoch [3/100], Loss: 1.6050
Epoch [4/100], Loss: 1.6023
Epoch [5/100], Loss: 1.5997
Epoch [6/100], Loss: 1.5972
Epoch [7/100], Loss: 1.5946
Epoch [8/100], Loss: 1.5919
Epoch [9/100], Loss: 1.5890
Epoch [10/100], Loss: 1.5860
Epoch [11/100], Loss: 1.5827
Epoch [12/100], Loss: 1.5793
Epoch [13/100], Loss: 1.5756
Epoch [14/100], Loss: 1.5717
Epoch [15/100], Loss: 1.5676
Epoch [16/100], Loss: 1.5633
Epoch [17/100], Loss: 1.5589
Epoch [18/100], Loss: 1.5544
Epoch [19/100], Loss: 1.5498
Epoch [20/100], Loss: 1.5451
Epoch [21/100], Loss: 1.5405
Epoch [22/100], Loss: 1.5360
Epoch [23/100], Loss: 1.5317
Epoch [24/100], Loss: 1.5277
Epoch [25/100], Loss: 1.5240
Epoch [26/100], Loss: 1.5204
Epoch [27/100], Loss: 1.5170
Epoch [28/100], Loss: 1.5136
Epoch [29/100], Loss: 1.5100
Epoch [30/100], Loss: 1.5062
Epoch [31/100], Loss: 1.5024
Epoch [32/100], Loss: 1.4985
Epoch [33/100], Loss: 1.4946
Epoch [34/100], Loss: 1.4910
Epoch [35/100], Loss: 1

In [29]:
loss_fn = nn.CrossEntropyLoss()
train_predictions = []
train_targets = []

model.eval()

size = len(train_dataloader.dataset)
num_batches = len(train_dataloader)
loss, correct = 0, 0

with torch.no_grad():
    for X, y, seq_lengths in train_dataloader:
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        pred = model(X, seq_lengths)
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_predictions.append(pred.argmax(1))
        train_targets.append(y)
        
loss /= num_batches
correct /= size
print(f"Training Error: \n Accuracy: {(correct):>0.2f}, Avg loss: {loss:>8f} \n")

/tmp/ipykernel_8843/4234377109.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


Training Error: 
 Accuracy: 0.43, Avg loss: 1.384185 



In [30]:
loss_fn = nn.CrossEntropyLoss()
val_predictions = []
val_targets = []

model.eval()

size = len(val_dataloader.dataset)
num_batches = len(val_dataloader)
loss, correct = 0, 0

with torch.no_grad():
    for X, y, seq_lengths in val_dataloader:
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)
        pred = model(X, seq_lengths)
        loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        val_predictions.append(pred.argmax(1))
        val_targets.append(y)

loss /= num_batches
correct /= size
print(f"Validation Error: \n Accuracy: {(correct):>0.2f}, Avg loss: {loss:>8f} \n")

/tmp/ipykernel_8843/1500241406.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)


Validation Error: 
 Accuracy: 0.43, Avg loss: 1.384186 



In [31]:
train_predictions = train_predictions[0].tolist()
train_targets = train_targets[0].tolist()

train_predictions = [x for x in train_predictions]
train_targets = [x for x in train_targets]

val_predictions = val_predictions[0].tolist()
val_targets = val_targets[0].tolist()

val_predictions = [x for x in val_predictions]
val_targets = [x for x in val_targets]

train_evaluation_results = evaluate(predictions=train_predictions, targets=train_targets, num_classes=5)
val_evaluation_results = evaluate(predictions=val_predictions, targets=val_targets, num_classes=5)

In [32]:
for metric, value in train_evaluation_results['aggregated_metrics'].items():
    print(f"{metric.capitalize()}: {value:.4f}")

print('\n')

for metric, value in val_evaluation_results['aggregated_metrics'].items():
    print(f"{metric.capitalize()}: {value:.4f}")

Accuracy: 0.4339
Micro_precision: 0.4339
Micro_recall: 0.4339
Micro_f1: 0.4339
Macro_precision: 0.4249
Macro_recall: 0.4337
Macro_f1: 0.4247


Accuracy: 0.4339
Micro_precision: 0.4339
Micro_recall: 0.4339
Micro_f1: 0.4339
Macro_precision: 0.4249
Macro_recall: 0.4337
Macro_f1: 0.4247


In [33]:
for i in range(5):
    print(f'\nClass {i}')
    for metric, value in train_evaluation_results['extended_metrics'].items():
        print(f'{metric.capitalize()}: {value[i]:.4f}')


Class 0
True_positives: 5499.0000
True_negatives: 27898.0000
False_positives: 5776.0000
False_negatives: 2890.0000
Precision: 0.4877
Recall: 0.6555
F1_score: 0.5593
Accuracy: 0.7940

Class 1
True_positives: 3037.0000
True_negatives: 29181.0000
False_positives: 4472.0000
False_negatives: 5373.0000
Precision: 0.4044
Recall: 0.3611
F1_score: 0.3816
Accuracy: 0.7659

Class 2
True_positives: 2085.0000
True_negatives: 29920.0000
False_positives: 3794.0000
False_negatives: 6264.0000
Precision: 0.3547
Recall: 0.2497
F1_score: 0.2931
Accuracy: 0.7609

Class 3
True_positives: 3601.0000
True_negatives: 28415.0000
False_positives: 5187.0000
False_negatives: 4860.0000
Precision: 0.4098
Recall: 0.4256
F1_score: 0.4175
Accuracy: 0.7611

Class 4
True_positives: 4028.0000
True_negatives: 29025.0000
False_positives: 4584.0000
False_negatives: 4426.0000
Precision: 0.4677
Recall: 0.4765
F1_score: 0.4720
Accuracy: 0.7858


In [34]:
for i in range(5):
    print(f'\nClass {i}')
    for metric, value in val_evaluation_results['extended_metrics'].items():
        print(f'{metric.capitalize()}: {value[i]:.4f}')


Class 0
True_positives: 5499.0000
True_negatives: 27898.0000
False_positives: 5776.0000
False_negatives: 2890.0000
Precision: 0.4877
Recall: 0.6555
F1_score: 0.5593
Accuracy: 0.7940

Class 1
True_positives: 3037.0000
True_negatives: 29181.0000
False_positives: 4472.0000
False_negatives: 5373.0000
Precision: 0.4044
Recall: 0.3611
F1_score: 0.3816
Accuracy: 0.7659

Class 2
True_positives: 2085.0000
True_negatives: 29920.0000
False_positives: 3794.0000
False_negatives: 6264.0000
Precision: 0.3547
Recall: 0.2497
F1_score: 0.2931
Accuracy: 0.7609

Class 3
True_positives: 3601.0000
True_negatives: 28415.0000
False_positives: 5187.0000
False_negatives: 4860.0000
Precision: 0.4098
Recall: 0.4256
F1_score: 0.4175
Accuracy: 0.7611

Class 4
True_positives: 4028.0000
True_negatives: 29025.0000
False_positives: 4584.0000
False_negatives: 4426.0000
Precision: 0.4677
Recall: 0.4765
F1_score: 0.4720
Accuracy: 0.7858
